In [1]:
# Dependencies
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests

In [2]:
# Credentials
from config import clientID, clientSEC
client_credentials_manager = SpotifyClientCredentials(client_id=clientID, client_secret=clientSEC)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
AUTH_URL = 'https://accounts.spotify.com/api/token'

In [ ]:
# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# Convert the response to JSON
auth_response_data = auth_response.json()

# Save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

BASE_URL = 'https://api.spotify.com/v1/'